In [413]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader
import re
import sys
from torch.nn.functional import normalize

In [391]:
proj_path = "/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand"
class_path = os.path.join(proj_path,"Thailand_classification")
ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
# pd.read_csv("
os.listdir(class_path)

loc_id = 0

s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
s2_csv_name

class_colname = 'Subclass2019'

proj_normpath = os.path.normpath(proj_path)
proj_dirname = proj_normpath.split(os.sep)[-1]
proj_name = re.sub("_classification$","",proj_dirname)
class_path = os.path.join(proj_path, proj_name + "_classification")
ts_path = os.path.join(proj_path, proj_name + "_download_timeseries")
pt_classes = pd.read_csv(os.path.join(class_path,"location_classification.csv"))
pt_classes = classes[['loc_id', class_colname]].dropna()

pt_classes

,loc_id,Subclass2019
0,0,Plantation
1,1,Crop(Single)
2,2,Crop(Single)
3,3,Crop(Single)
4,4,Plantation
...,...,...
496,496,Crop(Single)
497,497,Crop(Single)
498,498,Plantation
499,499,Plantation


In [399]:
# prep dataset
date_range = pd.to_datetime(['2019-06-01','2020-05-31'])

def prep_s2_loc(loc_id, date_range, proj_path):
    ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
    s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
    s2_csv_path = os.path.join(ts_path, s2_csv_name)
    s2_ts = pd.read_csv(s2_csv_path)

    # extract dates from image ids
    s2_ts['datestr'] = [re.sub("(^[0-9]+)[a-zA-Z].*","\\1",x) for x in s2_ts.image_id]
    s2_ts['date'] = pd.to_datetime(s2_ts.datestr, format = "%Y%m%d")

    # subset to cloud-free days AND within date_range
    s2_ts = s2_ts[(s2_ts.date >= date_range[0]) & (s2_ts.date <= date_range[1]) & (s2_ts.cloudmask == 0)]

    # calculate day from startday
    date_diff = (s2_ts.date - date_range[0])
    s2_ts['day'] = [x.days for x in date_diff]
    s2_ts['loc_id'] = loc_id

    # select only predictor and position columns, return tensor
    s2_ts_x = s2_ts[['loc_id','day','B8','B4','B3','B2']]
    return s2_ts_x

if os.path.exists(os.path.join(proj_path, 's2_ts_prepped.pt')):
    loc_ts_tor = torch.load(os.path.join(proj_path, 's2_ts_prepped.pt'))
    
else:
    s2_ts_list = []
    loc_id_list = []
    for i in np.arange(pt_classes.shape[0]):
        # loc_id = 499
        # print(loc_id)
        loc_id = pt_classes.loc_id.iloc[i]
        # loc_id_list.append(loc_id)
        s2_ts_loc = prep_s2_loc(loc_id, date_range, proj_path)
        s2_ts_tor = torch.tensor(s2_ts_loc.to_numpy())
        s2_ts_list.append(s2_ts_tor)
        
    loc_ts_tor = torch.cat(s2_ts_list)

    torch.save(loc_ts_tor, os.path.join(proj_path, 's2_ts_prepped.pt'))

sys.getsizeof(loc_ts_tor)


72

In [415]:
pt_idx = loc_ts_tor[:,0] == 40
loc_ts_tor[pt_idx]
loc_ts_tor.mean(dim = 1)

tensor([ 920.6667,  972.1667,  877.1667,  ..., 1228.3333, 1113.3333,
        1425.8333], dtype=torch.float64)

In [385]:
os.path.exists(os.path.join(proj_path, 's2_ts_prepped.pt'))

True

In [285]:
class s2Dataset(Dataset):
    """Sentinel 2 dataset"""
    
    def __init__(self, proj_path, class_colname):
        """
        Args:
            proj_path (string): path to manclassify project
        """
        self.proj_path = proj_path
        proj_normpath = os.path.normpath(proj_path)
        proj_dirname = proj_normpath.split(os.sep)[-1]
        self.proj_name = re.sub("_classification$","",proj_dirname)
        self.class_path = os.path.join(proj_path, self.proj_name + "_classification")
        self.ts_path = os.path.join(proj_path, self.proj_name + "_download_timeseries")
        self.pt_classes = pd.read_csv(os.path.join(self.class_path,"location_classification.csv"))
        self.pt_classes = classes[['loc_id', class_colname]].dropna()
        # self.pt_classes['loc_id'] = self.pt_classes['loc_id'] + 10.5 # for testing index only
        self.classes = pd.unique(self.pt_classes[class_colname])
        self.labels = self.pt_classes.assign(val = 1).pivot_table(columns = class_colname, index = 'loc_id', values = 'val', fill_value= 0)

    
    def __getitem__(self, idx):
        
        def
        date_range = pd.to_datetime(['2019-06-01','2020-05-31'])
        loc_id = self.labels.index[idx]
        self.last_loc_id = loc_id
        s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
        s2_csv_path = os.path.join(self.ts_path, s2_csv_name)
        s2_ts = pd.read_csv(s2_csv_path)
        
        # extract dates from image ids
        s2_ts['datestr'] = [re.sub("(^[0-9]+)[a-zA-Z].*","\\1",x) for x in s2_ts.image_id]
        s2_ts['date'] = pd.to_datetime(s2_ts.datestr, format = "%Y%m%d")
        
        # subset to cloud-free days AND within date_range
        s2_ts = s2_ts[(s2_ts.date >= date_range[0]) & (s2_ts.date <= date_range[1]) & (s2_ts.cloudmask == 0)]
        
        # calculate day from startday
        date_diff = (s2_ts.date - date_range[0])
        s2_ts['day'] = [x.days for x in date_diff]
        
        # select only predictor and position columns, return tensor
        s2_ts_x = s2_ts[['B8','B4','B3','B2','day']]
        x = torch.tensor(s2_ts_x.to_numpy())
        
        # get one-hot encoding for the point as tensor
        y = torch.tensor(self.labels.iloc[idx].to_numpy())
        
        return x, y
        
    def __len__(self):
        return self.pt_classes.shape[0]

In [335]:
proj_path = "/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand"
# date_rangeX = pd.to_datetime(['2019-06-01','2020-05-31'])
s2_dataset = s2Dataset(proj_path = proj_path, class_colname = 'Subclass2019')
x = s2_dataset.__getitem__(10)
sys.getsizeof(x)

56

In [ ]:
dataloader = DataLoader(s2_dataset, batch_size = 10, shuffle = True)



In [ ]:
len(dataloader)